In [53]:
# autoroald
%load_ext autoreload
%autoreload 2
from aitrading.ml_logic2 import data_engineering, feature_engineering, features_preprocessing, model_engineering,utility, repository

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
df = utility.load_data_source("/Users/kassraniroumand/code/aitrading/aitrading/data/eurousd_df_clean_2.csv")

In [55]:
df = feature_engineering.enrich_dataframe(df)

In [56]:
X_train, y_train, X_val, y_val, X_test, y_test = data_engineering.split_data(df)

In [57]:
X_train_reshaped, y_train_categorical, X_val_reshaped, y_val_categorical, X_test_reshaped, y_test_categorical = features_preprocessing.preprocess_features_and_target(X_train, y_train, X_val, y_val, X_test, y_test)

Training set size: 125775
Validation set size: 26951
Test set size: 26953


/Users/kassraniroumand/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [58]:
input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])
model = model_engineering.create_compile_model(input_shape)

/Users/kassraniroumand/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [59]:
history = model_engineering.train_model(model, X_train_reshaped, y_train_categorical, X_val_reshaped, y_val_categorical)

3931/3931 ━━━━━━━━━━━━━━━━━━━━ 31s 7ms/step - accuracy: 0.5094 - loss: 0.7242 - val_accuracy: 0.5114 - val_loss: 0.6929


In [60]:
repository.save_model_default_path(model)

✅ Saving model...
✅ Model saved at: /Users/kassraniroumand/code/aitrading/aitrading/models/model_2024-04-02-19-09-29.keras


In [61]:
model = repository.load_latest_model()

✅ Loading latest model: /Users/kassraniroumand/code/aitrading/aitrading/models/model_2024-04-02-19-09-29.keras


/Users/kassraniroumand/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 28 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 1 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>]

In [43]:
print(X_train_reshaped.shape)
print(y_train_categorical.shape)
print(X_val_reshaped.shape)
print(y_val_categorical.shape)
print(X_test_reshaped.shape)
print(y_test_categorical.shape)

(125775, 1, 11)
(125775,)
(26951, 1, 11)
(26951,)
(26953, 1, 11)
(26953,)


In [ ]:
model.evaluate(X_test_reshaped, y_test_categorical)

In [66]:
quote = utility.data_tradermade_source()
one_hour_df_for_prediction = feature_engineering.enrich_dataframe(quote, 'date')
# df_for_prediction = one_hour_df_for_prediction.iloc[-1]
one_hour_df_for_prediction

start 2024-04-02-17:16, now 2024-04-02-18:16


,close,date,high,low,open,timestamp,target,hour,day_of_week,sin_day,cos_day,price_change_5_intervals,rolling_avg_price_10_intervals,rolling_avg_price_10_close_intervals,ma_30m,ma_1h,ema_30min,ema_1h,rsi
11,1.07668,2024-04-02 18:11:00,1.07685,1.07664,1.07684,2024-04-02 18:11:00,0,18,1,0.781831,0.62349,0.00005,1.076750,1.076660,1.076750,1.076640,1.076722,1.076603,55.555556
12,1.07668,2024-04-02 18:16:00,1.07669,1.07666,1.07669,2024-04-02 18:16:00,0,18,1,0.781831,0.62349,0.00008,1.076766,1.076676,1.076766,1.076647,1.076710,1.076604,59.523810


In [65]:
import pandas as pd

model.predict(pd.DataFrame(one_hour_df_for_prediction.iloc[-1]).T)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.float64).